# Lab 12: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2022-05-02 18:04:38,169 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-02 18:04:48,148 | INFO     |:  | Number of observations per grid cell: 18
2022-05-02 18:04:48,154 | INFO     |:  | Precaching Views...


/Users/roninsharma/Desktop/Cornell/ECE 5960/Fast-Robots/Lab12/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-05-02 18:04:54,666 | INFO     |:  | Precaching Time: 6.511 secs
2022-05-02 18:04:54,667 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-02 18:04:54,673 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-02 18:04:57,763 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-02 18:04:57,765 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-05-02 18:05:00,688 | INFO     |: Update Step
2022-05-02 18:05:00,696 | INFO     |:      | Update Time: 0.006 secs
2022-05-02 18:05:00,699 | INFO     |: ---------- UPDATE STATS -----------
2022-05-02 18:05:00,725 | INFO     |: GT index      : (6, 4, 9)
2022-05-02 18:05:00,728 | INFO     |: Bel index     : (5, 3, 9) with prob = 0.6584420
2022-05-02 18:05:00,733 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-02 18:05:00,738 | INFO     |: GT            : (0.000, 0.000, 360.000)
2022-05-02 18:05:00,739 | INFO     |: Belief        : (0.000, -0.305, 10.000)
2022-05-02 18:05:00,741 | INFO     |: POS ERROR     : (-0.000, 0.305, 350.000)
2022-05-02 18:05:00,742 | INFO     |: ---------- UPDATE STATS -----------


In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2022-05-02 18:05:04,074 | INFO     |: Prediction Step
2022-05-02 18:05:04,237 | INFO     |:  | Prediction Time: 0.162 secs
2022-05-02 18:05:04,238 | INFO     |: ---------- PREDICTION STATS -----------
2022-05-02 18:05:04,253 | INFO     |: GT index         : (6, 3, 7)
2022-05-02 18:05:04,256 | INFO     |: Prior Bel index  : (6, 4, 7) with prob = 0.0930600
2022-05-02 18:05:04,259 | INFO     |: POS ERROR        : (-0.285, -0.005, -6.956)
2022-05-02 18:05:04,280 | INFO     |: ---------- PREDICTION STATS -----------
2022-05-02 18:05:07,185 | INFO     |: Update Step
2022-05-02 18:05:07,194 | INFO     |:      | Update Time: 0.008 secs
2022-05-02 18:05:07,195 | INFO     |: ---------- UPDATE STATS -----------
2022-05-02 18:05:07,212 | INFO     |: GT index      : (6, 3, 7)
2022-05-02 18:05:07,214 | INFO     |: Bel index     : (5, 4, 7) with prob = 0.9999979
2022-05-02 18:05:07,217 | INFO     |: Bel_bar prob at index = 2.5108452990789303e-05
2022-05-02 18:0

2022-05-02 18:05:38,333 | INFO     |: GT index      : (10, 1, 11)
2022-05-02 18:05:38,342 | INFO     |: Bel index     : (10, 1, 11) with prob = 0.5555975
2022-05-02 18:05:38,345 | INFO     |: Bel_bar prob at index = 0.0026434457936714826
2022-05-02 18:05:38,349 | INFO     |: GT            : (1.380, -0.840, 2200.874)
2022-05-02 18:05:38,357 | INFO     |: Belief        : (1.524, -0.914, 50.000)
2022-05-02 18:05:38,362 | INFO     |: POS ERROR     : (-0.144, 0.074, 2150.874)
2022-05-02 18:05:38,387 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


----------------- 6 -----------------
2022-05-02 18:05:40,993 | INFO     |: Prediction Step
2022-05-02 18:05:41,116 | INFO     |:  | Prediction Time: 0.122 secs
2022-05-02 18:05:41,117 | INFO     |: ---------- PREDICTION STATS -----------
2022-05-02 18:05:41,149 | INFO     |: GT index         : (10, 1, 12)
2022-05-02 18:05:41,154 | INFO     |: Prior Bel index  : (10, 3, 13) with prob = 0.1305286
2022-05-02 

2022-05-02 18:06:12,686 | INFO     |:  | Prediction Time: 0.158 secs
2022-05-02 18:06:12,689 | INFO     |: ---------- PREDICTION STATS -----------
2022-05-02 18:06:12,708 | INFO     |: GT index         : (7, 6, 3)
2022-05-02 18:06:12,709 | INFO     |: Prior Bel index  : (7, 7, 5) with prob = 0.1082851
2022-05-02 18:06:12,710 | INFO     |: POS ERROR        : (-0.178, -0.065, 4278.753)
2022-05-02 18:06:12,714 | INFO     |: ---------- PREDICTION STATS -----------
2022-05-02 18:06:15,624 | INFO     |: Update Step
2022-05-02 18:06:15,629 | INFO     |:      | Update Time: 0.005 secs
2022-05-02 18:06:15,630 | INFO     |: ---------- UPDATE STATS -----------
2022-05-02 18:06:15,635 | INFO     |: GT index      : (7, 6, 3)
2022-05-02 18:06:15,639 | INFO     |: Bel index     : (7, 7, 3) with prob = 1.0
2022-05-02 18:06:15,643 | INFO     |: Bel_bar prob at index = 0.004834086872465232
2022-05-02 18:06:15,647 | INFO     |: GT            : (0.432, 0.849, 4568.754)
2022-05-02 18:06:15,655 | INFO     |